In [41]:
using JuMP
#Import HiGHS solver
using HiGHS

#Create a JuMP model named picframe1 that will be solved using the HiGHS solver
coffeeblend = Model(HiGHS.Optimizer);
types = [:Columbian,:Brazilian,:Sumatran];
blends = [:Robust, :Light];

beansAviliable = Dict(:Columbian=> 550,:Brazilian=>450,:Sumatran=>650);
beansPrice=Dict(:Columbian=> 1,:Brazilian=>0.85,:Sumatran=>1.55);
coffPrice=Dict(:Robust=>4.25,:Light=>3.95)

#inf represent lowerbound; sup represent upper bound; setting upper lower bound for beans in each coffee
@variable(coffeeblend,x[types, blends]>=0);
@constraint(coffeeblend, RSInf, x[:Sumatran,:Robust] >= 0.6*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, RSSup, x[:Sumatran,:Robust] <= 0.75*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, RCInf, x[:Columbian,:Robust] >= 0.1*sum(x[i,:Robust] for i in types));
@constraint(coffeeblend, LBInf, x[:Brazilian,:Light] >= 0.5*sum(x[i,:Light] for i in types));
@constraint(coffeeblend, LBSup, x[:Brazilian,:Light] <= 0.6*sum(x[i,:Light] for i in types));
@constraint(coffeeblend, LSSup, x[:Sumatran,:Light] <= 0.6*sum(x[i,:Light] for i in types));

#setting bound for beans available
@constraint(coffeeblend, beansConstraint[i in types], sum(x[i,j] for j in blends)<=beansAviliable[i]);

#objective function with revenue-cost
@objective(coffeeblend,Max, sum(x[i,j] for i in types for j in blends)*coffPrice[j]-
sum(x[i,j] for i in types for j in blends)*beansPrice[i]);

print(coffeeblend);


Max 3.25 x[Columbian,Robust] + 2.95 x[Columbian,Light] + 3.4 x[Brazilian,Robust] + 3.1 x[Brazilian,Light] + 2.7 x[Sumatran,Robust] + 2.4000000000000004 x[Sumatran,Light]
Subject to
 RSInf : -0.6 x[Columbian,Robust] - 0.6 x[Brazilian,Robust] + 0.4 x[Sumatran,Robust] ≥ 0.0
 RCInf : 0.9 x[Columbian,Robust] - 0.1 x[Brazilian,Robust] - 0.1 x[Sumatran,Robust] ≥ 0.0
 LBInf : -0.5 x[Columbian,Light] + 0.5 x[Brazilian,Light] - 0.5 x[Sumatran,Light] ≥ 0.0
 RSSup : -0.75 x[Columbian,Robust] - 0.75 x[Brazilian,Robust] + 0.25 x[Sumatran,Robust] ≤ 0.0
 LBSup : -0.6 x[Columbian,Light] + 0.4 x[Brazilian,Light] - 0.6 x[Sumatran,Light] ≤ 0.0
 LSSup : -0.6 x[Columbian,Light] - 0.6 x[Brazilian,Light] + 0.4 x[Sumatran,Light] ≤ 0.0
 beansConstraint[Columbian] : x[Columbian,Robust] + x[Columbian,Light] ≤ 550.0
 beansConstraint[Brazilian] : x[Brazilian,Robust] + x[Brazilian,Light] ≤ 450.0
 beansConstraint[Sumatran] : x[Sumatran,Robust] + x[Sumatran,Light] ≤ 650.0
 x[Columbian,Robust] ≥ 0.0
 x[Brazilian,Robust

In [42]:
optimize!(coffeeblend);
@show objective_value(coffeeblend);
@show value.(x);


Presolving model
9 rows, 6 cols, 24 nonzeros
9 rows, 6 cols, 24 nonzeros
Presolve : Reductions: rows 9(-0); columns 6(-0); elements 24(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -4.4549962997e+01 Ph1: 8(11.475); Du: 6(44.55) 0s
          5    -4.9025000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 5
Objective value     :  4.9025000000e+03
HiGHS run time      :          0.00
objective_value(coffeeblend) = 4902.5
value.(x) = 2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:Columbian, :Brazilian, :Sumatran]
    Dimension 2, [:Robust, :Light]
And data, a 3×2 Matrix{Float64}:
 323.33333333333337  226.66666666666663
 109.99999999999996  339.9999999999999
 650.0                 0.0
